In [128]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Bidirectional, LSTM
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
import sklearn

In [3]:
# Read data from csv file, consider column 'date' as index and parse it into date object
dataset = pd.read_csv('../input/historical_stock_prices.csv', index_col='date', parse_dates=['date'])

In [153]:
trainSet = dataset['2017':'2017'].sort_values(by=['ticker','date'])

In [154]:
trainSet.head()

,ticker,open,close,adj_close,low,high,volume
date,,,,,,,
2017-01-03,A,45.930000,46.490002,45.861324,45.740002,46.750000,1739600
2017-01-04,A,46.930000,47.099998,46.463070,46.820000,47.380001,1821300
2017-01-05,A,47.049999,46.540001,45.910652,46.360001,47.070000,1503700
2017-01-06,A,46.630001,47.990002,47.341042,46.560001,48.070000,2883400
2017-01-09,A,48.009998,48.139999,47.489006,47.910000,48.560001,2575300


# Normalize each symbols independently

In [155]:
symbols = trainSet.ticker.unique()

In [157]:
symbols = symbols[:10]
print(len(symbols))

10


In [158]:
x_train = []
y_train = []
scaler = {}
timesteps = 60
for s in symbols:
    # normalize data
    sc = MinMaxScaler(feature_range=(0,1))     
    train_stock = trainSet.loc[trainSet['ticker'] == s][['adj_close']].values
    train_stock_scaled = sc.fit_transform(train_stock)
    scaler[s] = sc
    # Prepare x_train y_train
    l = len(train_stock_scaled)
    for i in range(timesteps, l):
        x_train.append(train_stock_scaled[i-timesteps: i, 0])
        y_train.append(train_stock_scaled[i, 0])
        
X_train, Y_train = np.array(x_train), np.array(y_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [159]:
X_train.shape, Y_train.shape

((1889, 60, 1), (1889,))

In [161]:
def create_model(n_neurons = 2, dropout_rate = 0.3):
    regressorGRU = Sequential()
    
    # First layer
    regressorGRU.add(GRU(units=n_neurons, input_shape=(60,1), activation = 'tanh'))
    regressorGRU.add(Dropout(dropout_rate))
    
    # Top layer
    regressorGRU.add(Dense(units=1))
    
    #Compile model
    regressorGRU.compile(loss='mean_squared_error', 
                         optimizer = 'rmsprop', metrics = ['accuracy'])
    
    return regressorGRU

In [164]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Hyper parameters 
n_neurons = [20, 30,32,60,64,80]
dropout_rate = [0.1, 0.2,0.3]


model = KerasClassifier(build_fn=create_model)

param_grid = dict(n_neurons=n_neurons, dropout_rate = dropout_rate)

# Hyper parameters optimization using Grid search
grid = GridSearchCV(estimator = model, 
                    param_grid = param_grid, 
                    cv = 2, 
                    verbose=1, 
                    n_jobs=1)

grid_result = grid.fit(X_train, Y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/1
945/945 [==============================] - 8s 8ms/step
Epoch 1/1
944/944 [==============================] - 8s 8ms/step
Epoch 1/1
945/945 [==============================] - 8s 8ms/step
Epoch 1/1
944/944 [==============================] - 8s 8ms/step
Epoch 1/1
945/945 [==============================] - 8s 8ms/step
Epoch 1/1
944/944 [==============================] - 8s 8ms/step
Epoch 1/1
945/945 [==============================] - 8s 8ms/step
Epoch 1/1
944/944 [==============================] - 8s 8ms/step
Epoch 1/1
945/945 [==============================] - 9s 10ms/step
Epoch 1/1
944/944 [==============================] - 9s 10ms/step
Epoch 1/1
945/945 [==============================] - 9s 10ms/step
Epoch 1/1
944/944 [==============================] - 9s 9ms/step
Epoch 1/1
945/945 [==============================] - 9s 10ms/step
Epoch 1/1
944/944 [==============================] - 9s 9ms/step
Epoch 1/1
945/945 [==============================] - 9s 10ms/step
Epoch 1/1
944/944 [=

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 18.9min finished


Epoch 1/1
1889/1889 [==============================] - 26s 14ms/step - loss: 854152.1899 - acc: 0.0011


In [166]:
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  0.008999470619375331
Best Params:  {'dropout_rate': 0.3, 'n_neurons': 30}
